In [1]:
print('env hola')

env hola


In [2]:
# Cargar los datos en local

import pandas as pd
df = pd.read_csv('data/interacciones.csv')

print(df.head())
print(df.info())

         ID  USER_ID WORKFLOW_STATE FORCED_READ_STATE  DISCUSSION_ENTRY_ID  \
0  19033728   191041           read               NaN               890724   
1  19033727    97827           read               NaN               599902   
2  19033729   200825           read               NaN               639432   
3  19468980   166609           read               NaN               893052   
4  19468982   169487           read               NaN               798588   

   RATING                     META_TS                    TS_CARGA  \
0     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
1     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
2     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
3     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   
4     NaN  2024-03-28 19:41:48.802000  2025-02-18 14:14:46.140000   

              TS_PROCESSAMENT  IND_LAST  \
0  2025-02-18 14:55:59.754000         0   
1  2025-02-18 14:55:59.754000 

In [3]:
# Limpieza previa

df.drop(columns=['IND_LAST', 'METADATA_FILENAME', 'METADATA_FILE_LAST_MODIFIED', 'META_TS', 'TS_CARGA', 'TS_PROCESSAMENT'], inplace=True)

print(df.isnull().sum())
df.fillna(0, inplace=True)

df['USER_ID'] = df['USER_ID'].astype(int)
df['DISCUSSION_ENTRY_ID'] = df['DISCUSSION_ENTRY_ID'].astype(int)
df['RATING'] = df['RATING'].astype(int)
df['FORCED_READ_STATE'] = df['FORCED_READ_STATE'].astype(bool)

print(df.head())

ID                        0
USER_ID                   0
WORKFLOW_STATE            0
FORCED_READ_STATE      8075
DISCUSSION_ENTRY_ID       0
RATING                 9950
dtype: int64
         ID  USER_ID WORKFLOW_STATE  FORCED_READ_STATE  DISCUSSION_ENTRY_ID  \
0  19033728   191041           read              False               890724   
1  19033727    97827           read              False               599902   
2  19033729   200825           read              False               639432   
3  19468980   166609           read              False               893052   
4  19468982   169487           read              False               798588   

   RATING  
0       0  
1       0  
2       0  
3       0  
4       0  


Los nodos son los usuarios (USER_ID).
Las aristas representan interacciones (DISCUSSION_ENTRY_ID).
El peso de las aristas se calcula en función del RATING. (pero estan todos nulos)

In [4]:
df['RATING'].value_counts()

RATING
0    9955
1      45
Name: count, dtype: int64

In [5]:
import networkx as nx

G = nx.DiGraph()

for _, row in df.iterrows():
    user_from = row['USER_ID']
    discussion_id = row['DISCUSSION_ENTRY_ID']
    weight = row['RATING']
    
    #nodo y arista
    G.add_node(user_from)
    G.add_node(discussion_id)
    G.add_edge(user_from, discussion_id, weight=weight)

print(f"Nodos: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}")


Nodos: 12203, Aristas: 10000


In [6]:
import matplotlib.pyplot as plt

#grafo
plt.figure(figsize=(15, 15))
pos = nx.spring_layout(G, k=0.3)
nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue', font_size=5, edge_color='yellow')
plt.title("Grafo de Interacciones")
plt.show()

# from pyvis.network import Network
# import networkx as nx

# # Crear el grafo
# net = Network(notebook=True, height="800px", width="100%", bgcolor="#222222", font_color="white")

# # Agregar nodos y aristas desde el grafo de NetworkX
# for node in G.nodes():
#     net.add_node(node, label=str(node), color="skyblue")

# for edge in G.edges():
#     net.add_edge(edge[0], edge[1], color="yellow")

# # Guardar y mostrar en el navegador
# net.show("grafo_interactivo.html")


KeyboardInterrupt: 

<Figure size 1500x1500 with 0 Axes>

In [ ]:
pos = nx.kamada_kawai_layout(G)
degree_centrality = nx.degree_centrality(G)
node_sizes = [1000 * degree_centrality[node] for node in G.nodes]
node_colors = [degree_centrality[node] for node in G.nodes]
pos = nx.spring_layout(G, k=0.1)  # Prueba otros valores de k
import matplotlib.pyplot as plt
import networkx as nx

# Crear el grafo (suponiendo que G es tu grafo)
# Puedes usar otros tipos de layout
pos = nx.spring_layout(G, k=0.1, iterations=50)  # Ajuste de layout
degree_centrality = nx.degree_centrality(G)
node_sizes = [1000 * degree_centrality[node] for node in G.nodes]
node_colors = [degree_centrality[node] for node in G.nodes]

plt.figure(figsize=(15, 15))
nx.draw(G, pos, with_labels=True, node_size=node_sizes, node_color=node_colors, font_size=10, edge_color='yellow', cmap=plt.cm.Blues)
plt.title("Grafo de Interacciones con Centralidad de Grado")
plt.show()


c:\Users\asanchezbelb\.conda\envs\hola\lib\site-packages\networkx\drawing\layout.py:726: RuntimeWarning: divide by zero encountered in divide
  costargs = (np, 1 / (dist_mtx + np.eye(dist_mtx.shape[0]) * 1e-3), meanwt, dim)


In [19]:
# from IPython.display import display, HTML
# display(HTML("grafo_interactivo.html"))


In [11]:
plt.savefig('grafo_interacciones.png')


<Figure size 640x480 with 0 Axes>

Metricas

Grado de Centralidad
Centralidad de Intermediación (Betweenness)
Centralidad de Cercanía (Closeness)

In [12]:
# Grado de Centralidad
degree_centrality = nx.degree_centrality(G)
print("\nTop 10 usuarios por Grado de Centralidad:")
print(sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

# Centralidad de Intermediación
betweenness_centrality = nx.betweenness_centrality(G)
print("\nTop 10 usuarios por Intermediación:")
print(sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

# Centralidad de Cercanía
closeness_centrality = nx.closeness_centrality(G)
print("\nTop 10 usuarios por Cercanía:")
print(sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])



Top 10 usuarios por Grado de Centralidad:
[(16087, 0.023602688083920666), (126386, 0.012456974266513685), (75367, 0.006802163579741026), (182525, 0.006146533355187674), (55543, 0.005818718242910997), (82112, 0.005326995574495984), (120839, 0.004671365349942632), (97827, 0.004425504015735125), (162295, 0.004261596459596787), (180194, 0.00401573512538928)]

Top 10 usuarios por Intermediación:
[(191041, 0.0), (890724, 0.0), (97827, 0.0), (599902, 0.0), (200825, 0.0), (639432, 0.0), (166609, 0.0), (893052, 0.0), (169487, 0.0), (798588, 0.0)]

Top 10 usuarios por Cercanía:
[(890381, 0.000245861334207507), (698812, 0.000245861334207507), (890734, 0.000245861334207507), (733781, 0.000245861334207507), (897347, 0.00016390755613833797), (905128, 0.00016390755613833797), (905069, 0.00016390755613833797), (899586, 0.00016390755613833797), (900398, 0.00016390755613833797), (900265, 0.00016390755613833797)]


In [15]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'browser'

#métricas
centrality_df = pd.DataFrame({
    'User': list(degree_centrality.keys()),
    'DegreeCentrality': list(degree_centrality.values()),
    'Betweenness': list(betweenness_centrality.values()),
    'Closeness': list(closeness_centrality.values())
})

fig = px.bar(centrality_df.sort_values(by='DegreeCentrality', ascending=False).head(10), 
             x='User', y='DegreeCentrality',
             title="Top 10 Usuarios por Grado de Centralidad",
             labels={'DegreeCentrality': 'Centralidad de Grado', 'User': 'Usuario'},
             color='DegreeCentrality')
fig.show()
